In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os,time,pickle
from tqdm import tqdm

from utils import data_process, visualize
from utils.utils import train, test, summary, setup_seed

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [2]:
data = np.load("./data_8/data_train.npy", allow_pickle=True)
x_train = data.item().get("x_train")
y_train = data.item().get("y_train")
data = np.load("./data_8/data_test.npy", allow_pickle=True)
x_test = data.item().get("x_test")
y_test = data.item().get("y_test")
x_train=x_train.transpose((0,2,1))
x_test=x_test.transpose((0,2,1))


x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()
traindata = TensorDataset(x_train, y_train)
testdata = TensorDataset(x_test, y_test)

train_loader = DataLoader(traindata, batch_size=128, shuffle=True)
test_loader = DataLoader(testdata, batch_size=256, shuffle=True)

In [17]:
class CRNN(nn.Module):
    def __init__(self, ):
        super(CRNN, self).__init__()
        self.conv1 = nn.Conv1d(
            in_channels=3,  out_channels=64, kernel_size=7, stride=2)
        self.conv2 = nn.Conv1d(
            in_channels=64,  out_channels=64, kernel_size=7, stride=2)
        self.pool1 = nn.MaxPool1d(kernel_size=4, stride=4)
        self.fla = nn.Flatten()
        self.dp = nn.Dropout(p=0.5)
        self.rnn = nn.RNN(input_size=3136, hidden_size=100)
        self.fc1 = nn.Linear(100, 2)

    def forward(self, data):
        out = F.relu(self.conv1(data))
        out = F.relu(self.conv2(out))     
        out = self.dp(out)
        out = self.pool1(out)
        out = self.fla(out)
        out = out.view(-1, 1, out.shape[-1])
        out, _ = self.rnn(out)
        out = self.fla(out)
        out = self.fc1(out)
        return F.log_softmax(out, dim=1)

In [18]:
data=torch.rand(16,3,800)
model = CRNN()
model(data).shape

torch.Size([16, 2])

In [19]:
%%time
setup_seed(42)
model = CRNN().cuda()
# model=TFEQ(channel=3, time_in=200).cuda()
L = []
test_hist = []
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, 200 + 1):
    start = time.time()
    loss = train(model, train_loader, optimizer, epoch)
    L.append(loss)
    print("time {:.1f} sec:".format(time.time() - start))
    acc = test(model, test_loader)
    test_hist.append(acc)

Train Epoch: 1, Average Loss: 0.3587
time 1.5 sec:
Test set: Average loss: 0.3637, Accuracy: 1837/2068 (88.83%)

Train Epoch: 2, Average Loss: 0.3491
time 1.4 sec:
Test set: Average loss: 0.3506, Accuracy: 1837/2068 (88.83%)

Train Epoch: 3, Average Loss: 0.3279
time 1.5 sec:
Test set: Average loss: 0.3663, Accuracy: 1900/2068 (91.88%)

Train Epoch: 4, Average Loss: 0.1815
time 1.3 sec:
Test set: Average loss: 0.1719, Accuracy: 1963/2068 (94.92%)

Train Epoch: 5, Average Loss: 0.1161
time 1.3 sec:
Test set: Average loss: 0.1293, Accuracy: 2020/2068 (97.68%)

Train Epoch: 6, Average Loss: 0.0766
time 1.3 sec:
Test set: Average loss: 0.1068, Accuracy: 2024/2068 (97.87%)

Train Epoch: 7, Average Loss: 0.0587
time 1.3 sec:
Test set: Average loss: 0.0745, Accuracy: 2024/2068 (97.87%)

Train Epoch: 8, Average Loss: 0.0511
time 1.4 sec:
Test set: Average loss: 0.0952, Accuracy: 2025/2068 (97.92%)

Train Epoch: 9, Average Loss: 0.0494
time 1.4 sec:
Test set: Average loss: 0.0685, Accuracy: 203

Train Epoch: 73, Average Loss: 0.0104
time 1.4 sec:
Test set: Average loss: 0.0303, Accuracy: 2046/2068 (98.94%)

Train Epoch: 74, Average Loss: 0.0138
time 1.4 sec:
Test set: Average loss: 0.0179, Accuracy: 2057/2068 (99.47%)

Train Epoch: 75, Average Loss: 0.0091
time 1.4 sec:
Test set: Average loss: 0.0195, Accuracy: 2056/2068 (99.42%)

Train Epoch: 76, Average Loss: 0.0110
time 1.4 sec:
Test set: Average loss: 0.0404, Accuracy: 2046/2068 (98.94%)

Train Epoch: 77, Average Loss: 0.0084
time 1.5 sec:
Test set: Average loss: 0.0222, Accuracy: 2059/2068 (99.56%)

Train Epoch: 78, Average Loss: 0.0075
time 1.4 sec:
Test set: Average loss: 0.0223, Accuracy: 2057/2068 (99.47%)

Train Epoch: 79, Average Loss: 0.0068
time 1.4 sec:
Test set: Average loss: 0.0271, Accuracy: 2054/2068 (99.32%)

Train Epoch: 80, Average Loss: 0.0103
time 1.5 sec:
Test set: Average loss: 0.0239, Accuracy: 2050/2068 (99.13%)

Train Epoch: 81, Average Loss: 0.0096
time 1.5 sec:
Test set: Average loss: 0.0225, Accu

Train Epoch: 145, Average Loss: 0.0048
time 1.5 sec:
Test set: Average loss: 0.0163, Accuracy: 2061/2068 (99.66%)

Train Epoch: 146, Average Loss: 0.0075
time 1.5 sec:
Test set: Average loss: 0.0288, Accuracy: 2055/2068 (99.37%)

Train Epoch: 147, Average Loss: 0.0037
time 1.5 sec:
Test set: Average loss: 0.0210, Accuracy: 2057/2068 (99.47%)

Train Epoch: 148, Average Loss: 0.0030
time 1.5 sec:
Test set: Average loss: 0.0291, Accuracy: 2052/2068 (99.23%)

Train Epoch: 149, Average Loss: 0.0041
time 1.5 sec:
Test set: Average loss: 0.0206, Accuracy: 2060/2068 (99.61%)

Train Epoch: 150, Average Loss: 0.0049
time 1.5 sec:
Test set: Average loss: 0.0321, Accuracy: 2053/2068 (99.27%)

Train Epoch: 151, Average Loss: 0.0035
time 1.5 sec:
Test set: Average loss: 0.0299, Accuracy: 2053/2068 (99.27%)

Train Epoch: 152, Average Loss: 0.0022
time 1.4 sec:
Test set: Average loss: 0.0317, Accuracy: 2055/2068 (99.37%)

Train Epoch: 153, Average Loss: 0.0030
time 1.4 sec:
Test set: Average loss: 0.0

In [20]:
model.eval()
y_pred = []
out=[]
y_test = []
with torch.no_grad():
    for data, target in test_loader:
        data = data.cuda()
        output = model(data)
        output=F.softmax(output, dim=1)
        pred = output.max(1, keepdim=True)[1]  
        pred = pred.cpu().numpy().squeeze()
        y_pred.append(pred)
        out.append(output.cpu().numpy())
        y_test.append(target.numpy().squeeze())
y_pred=np.hstack(y_pred)
y_test=np.hstack(y_test)
out=np.vstack(out)

summary(y_pred,y_test)

confusion matrix:
 [[1836   11]
 [   1  220]]
Roc score: 0.9948
F1 score: 0.9735
Recall_score: 0.9955
Precision_score: 0.9524
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      1847
           1       0.95      1.00      0.97       221

    accuracy                           0.99      2068
   macro avg       0.98      0.99      0.99      2068
weighted avg       0.99      0.99      0.99      2068



In [21]:
torch.save(model, './models/CRNN.pth')

In [6]:
earth=np.where(y_pred==1)[0]
non_earth=np.where(y_pred==0)[0]

p_earth=out[earth,1].squeeze()

p_non_earth=out[non_earth,0].squeeze()

In [7]:
a = np.random.rand(100)
hist,bins = np.histogram(p_earth,bins=10,range=(0,1))
print(hist)
print(bins)


[  0   0   0   0   0   0   2   1   2 232]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [8]:
a = np.random.rand(100)
hist,bins = np.histogram(p_non_earth,bins=10,range=(0,1))
print(hist)
print(bins)

[   0    0    0    0    0    1    1    2    2 1827]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
